In [1]:
%load_ext lab_black
%cd ../../..

/mnt/h/hev/log-analytics


In [2]:
import torch
import numpy as np
import pandas as pd
import pickle
from tqdm import tqdm
from collections import defaultdict
import sys

In [29]:
df = pd.read_csv("data/ori/test.csv")

In [3]:
distdeck = torch.load(
    "results/distilbert-base-uncased-ver8/distilbert-base-uncased-focal-AdamW-lr1e-05-ver8-os10_4-distdeck.pth"
)

In [4]:
deck2 = torch.load("results/distilbert-base-uncased-ver7/distilbert-base-uncased-focal-AdamW-lr1e-05-ver7-os10_3-deck2.pth")

In [5]:
distdeck["dist"].shape, distdeck["level"].shape

(torch.Size([1095951, 4]), torch.Size([1095951, 4]))

In [6]:
distdeck.keys(), deck2.keys()

(dict_keys(['dist', 'level', 'ids']),
 dict_keys(['feat', 'otext', 'fclevel', 'ids']))

In [7]:
total_len = 1418916

In [20]:
def policy(dists, tlevels, fclevel):
    if fclevel in [6, 4, 2]:
        return fclevel.item()
    if (tlevels == 5).all():
        return 5 if dists[0] < 2.0 else 7
    if (tlevels == 3).all():
        return 3 if dists[0] < 2.0 else 7
    if dists[0] < 1.0:
        # return tlevels[0].item()
        return fclevel.item()
    return 7

In [21]:
out_dists = [None for _ in range(total_len)]
out_levels = [None for _ in range(total_len)]
out_fclevels = [None for _ in range(total_len)]
N = distdeck["dist"].size(0)
with tqdm(total=N, ncols=100, file=sys.stdout) as t:
    for i in range(N):
        dists = distdeck["dist"][i]
        levels = distdeck["level"][i]
        fclevel = deck2["fclevel"][i]
        out_level = policy(dists, levels, fclevel)
        ids = deck2["ids"][i]
        for j in ids:
            out_levels[j - 1000000] = out_level
            out_dists[j - 1000000] = dists
            out_fclevels[j - 1000000] = fclevel
        t.update()

100%|██████████████████████████████████████████████████| 1095951/1095951 [01:35<00:00, 11480.98it/s]


In [22]:
out_levels = np.array(out_levels)

In [23]:
# 각 레벨별 개수 threshold 20
for i in range(8):
    cnt = (out_levels == i).sum()
    print(i, ":", cnt, f"{cnt / len(out_levels)*100:.2f}%")

0 : 1002507 70.65%
1 : 394339 27.79%
2 : 42 0.00%
3 : 12861 0.91%
4 : 34 0.00%
5 : 6117 0.43%
6 : 31 0.00%
7 : 2985 0.21%


In [24]:
(out_levels == 7).nonzero()

(array([     12,     104,     470, ..., 1415900, 1416792, 1417424]),)

In [30]:
df.full_log[104]

'E: probe_rpminfo: RPM: db5 error(-30973) from dbenv->failchk: BDB0087 DB_RUNRECOVERY: Fatal error, run database recovery'

In [27]:
out_dists[12]

tensor([1.5610, 2.2574, 3.5356, 3.5924])

In [19]:
distdeck["dist"].mean()

tensor(0.1104)

In [38]:
out_ids = list(range(1000000, 1000000 + len(out_levels)))

In [40]:
out_df = pd.DataFrame({"id": out_ids, "level": out_levels})

In [41]:
out_df

,id,level
0,1000000,0
1,1000001,0
2,1000002,1
3,1000003,0
4,1000004,1
...,...,...
1418911,2418911,0
1418912,2418912,0
1418913,2418913,1
1418914,2418914,0


In [42]:
out_df.to_csv(
    "results/distilbert-base-uncased/distilbert-base-uncased-focal-AdamW-lr2e-05-dsver5-os40_2-out_ver1.csv", index=False
)